Scikit-learn harjoitus 2
======================

Tämän harjoituksen tarkoitus on opettaa hieman edistyksellisimpiä toimintoja.



## Regressio
Regressio-ongelma:

TODO: selitys

Kokeile ainakin [näistä](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model)

In [42]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import datasets
from sklearn.model_selection import train_test_split

data = datasets.fetch_california_housing()
X = data.data
y = data.target
print("Datan koko: ", len(y))
print(data.DESCR)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # TODO: None

Datan koko:  20640
.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which 

In [45]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error, r2_score

regr = Lasso()#SGDRegressor()# LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

print('Coefficients: \n', regr.coef_)
print("Mean squared error: {:.2f}".format(mean_squared_error(y_test, y_pred)))

Coefficients: 
 [ 1.48196324e-01  5.72821070e-03  0.00000000e+00 -0.00000000e+00
 -8.16437293e-06 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
Mean squared error: 0.94


## Kuvien käyttäminen datana
Aloita lataamalla harjoitusdata. Tällä kertaa käytetään [MNIST](https://en.wikipedia.org/wiki/MNIST_database)-aineistoa,
joka sisältää käsinkirjoitettuja numeroita 0-9 8x8 pikselin kuvina.

In [1]:
import matplotlib.pyplot as plt
from sklearn import datasets
import numpy as np
dataset = datasets.load_digits()

# Näytetään ensimmäinen kuva
plt.imshow(dataset.images[0], cmap=plt.cm.gray_r, interpolation='nearest')
plt.title('Esimerkkikuva')
plt.show()

print("Esimerkkikuva pikseliarvoina: \n", dataset.images[0])

# dataset-olio tarjoaa havainnot myös valmiiksi prosessoituna yhdeksi vektoriksi/listaksi
print("Esimerkkikuva havaintovektorina: \n", dataset.data[0])


<Figure size 640x480 with 1 Axes>

Esimerkkikuva pikseliarvoina: 
 [[ 0.  0.  5. 13.  9.  1.  0.  0.]
 [ 0.  0. 13. 15. 10. 15.  5.  0.]
 [ 0.  3. 15.  2.  0. 11.  8.  0.]
 [ 0.  4. 12.  0.  0.  8.  8.  0.]
 [ 0.  5.  8.  0.  0.  9.  8.  0.]
 [ 0.  4. 11.  0.  1. 12.  7.  0.]
 [ 0.  2. 14.  5. 10. 12.  0.  0.]
 [ 0.  0.  6. 13. 10.  0.  0.  0.]]
Esimerkkikuva havaintovektorina: 
 [ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0. 13. 15. 10. 15.  5.  0.  0.  3.
 15.  2.  0. 11.  8.  0.  0.  4. 12.  0.  0.  8.  8.  0.  0.  5.  8.  0.
  0.  9.  8.  0.  0.  4. 11.  0.  1. 12.  7.  0.  0.  2. 14.  5. 10. 12.
  0.  0.  0.  0.  6. 13. 10.  0.  0.  0.]


Jotta kuvista, eli n*m-taulukoista päästään yksiulotteiseen havaintovektorimuotoon, ne pitää prosessoida.

In [2]:
n_samples = len(dataset.images)
data = dataset.images.reshape((n_samples, -1))
if np.array_equal(data, dataset.data):
    print("Kuvat ovat nyt yksiulotteisia")

Kuvat ovat nyt yksiulotteisia


Nyt edellistä harjoitusta hyväksi käyttäen suorita vaiheet ja luokittele malli haluamallasi algoritmilla 
ja parametreilla. Yritä päästä yleiseen tarkkuuteen OA>=0.9.

Jaa luomasi X ja luokat y opetus- ja testijoukkoihin `X_train, X_test, y_train ja  y_test` siten, 
että testijoukon osuus on 20% ja opetusjoukon osuus on 80% havainnoista. 
Käytä jaossa `random_state`-parametrin arvona 42

In [3]:
from sklearn.model_selection import train_test_split
X = data
y = dataset.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # TODO: None

In [ ]:
# Opeta malli ja luo ennustukset

In [4]:
# Evaluoi
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from utils import plot_confusion_matrix

## Epätasapainon korjaaminen
Jos luokkien määrä havaintojen välillä on epätasapainossa, malli suosii helposti luokkia, joissa on enemmän havaintoja, kuin toisissa. Tämä on yksi koneoppimisen suurista ongelmista. Yksi keino hallita ongelmaa, on havaintojen resamplaus, eli havaintojen määrän muuttaminen opetusjoukossa siten, että luokkien välinen epätasapaino pienenisi.

Tähän on kaksi menetelmää: 
* Havaintojen lisääminen pienimpiin luokkiin
* Havaintojen poistaminen suurimmista luokista


TODO: lisää tekstiä!

In [17]:
from imblearn.datasets import make_imbalance
from collections import Counter, OrderedDict

sampling_strategy = {0: 50, 1: 154, 2: 144, 3: 70, 4: 135, 5: 135, 6: 80, 7: 145, 8: 144, 9: 60}
X_train2, y_train2 =  make_imbalance(X_train, y_train, sampling_strategy=sampling_strategy)

print(dict(OrderedDict(sorted(Counter(y_train).items(), key=lambda t: t[0]))))
print(dict(OrderedDict(sorted(Counter(y_train2).items(), key=lambda t: t[0]))))

from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train, y_train)
print("Alkuperäinen OA: {:.2f}".format(clf.score(X_test, y_test)))

clf.fit(X_train2, y_train2)
print("Epätasapainoisen OA: {:.2f}".format(clf.score(X_test, y_test)))

{0: 145, 1: 154, 2: 144, 3: 149, 4: 135, 5: 135, 6: 146, 7: 145, 8: 144, 9: 140}
OrderedDict([(0, 50), (1, 154), (2, 144), (3, 70), (4, 135), (5, 135), (6, 80), (7, 145), (8, 144), (9, 60)])


/home/joona/IdeaProjects/MLCourse/venv/forNb/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Alkuperäinen OA: 0.47


/home/joona/IdeaProjects/MLCourse/venv/forNb/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Epätasapainoisen OA: 0.31


In [19]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


ros = RandomOverSampler()
X_train_oversampled, y_train_oversampled = ros.fit_resample(X_train2, y_train2)
rus = RandomUnderSampler()
X_train_undersampled, y_train_underrsampled = rus.fit_resample(X_train2, y_train2)

print(dict(OrderedDict(sorted(Counter(y_train_oversampled).items(), key=lambda t: t[0]))))
print(dict(OrderedDict(sorted(Counter(y_train_underrsampled).items(), key=lambda t: t[0]))))

clf = SVC()
clf.fit(X_train_oversampled, y_train_oversampled)
print("Oversampled OA: {:.2f}".format(clf.score(X_test, y_test)))

clf.fit(X_train_undersampled, y_train_underrsampled)
print("Undersampled OA: {:.2f}".format(clf.score(X_test, y_test)))

{0: 154, 1: 154, 2: 154, 3: 154, 4: 154, 5: 154, 6: 154, 7: 154, 8: 154, 9: 154}
{0: 50, 1: 50, 2: 50, 3: 50, 4: 50, 5: 50, 6: 50, 7: 50, 8: 50, 9: 50}


/home/joona/IdeaProjects/MLCourse/venv/forNb/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Alkuperäinen OA: 0.32


/home/joona/IdeaProjects/MLCourse/venv/forNb/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Epätasapainoisen OA: 0.54


## Ristivalidointi
Myös testijoukon ylisovitus on mahdollista kun mallia säädetään paljon. Tällöin on mahdollista käyttää kolmatta validointijoukkoa, mutta ristivalidointi on toinen (ja monesti parempi) vaihtoehto. Siinä käytetään hyväksi vain opetusjoukkoa, joka jaetaan osiin siten, että vuorollaan jokainen havainto on ristivalidoinnin sisäisessä opetusjoukossa ja vuorollaan testijoukossa. Näitä kutsutaan jaoiksi (engl. split). Tällöin oikeaa testijoukkoa voi säästää aivan viimeiseen säädetyn mallin validointiin saakka.

Suorita alla yksinkertainen ristivalidointi käyttäen [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score)-metodia ja haluamaasi luokittelualgoritmia. Aseta `CV`-parametri arvoksi 5 ja `n_jobs`-parametri arvoon -1.

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
clf = SVC(gamma=0.01)
scores = cross_val_score(clf, X_train, y_train, cv=5, n_jobs=-1)
print("Pisteet ovat:")
for oa in scores:
    print("    OA: {:.2f}".format(oa))

Pisteet ovat:
    OA: 0.74
    OA: 0.75
    OA: 0.76
    OA: 0.76
    OA: 0.73


`n_jobs`-parametri mahdollisti rinnakkaisen laskennan eri mallien välille. Tämä parametri löytyy jokaisesta ristivalidointia käyttävästä metodista ja myös joistain luokittelijoista, kuten Random Forest. Sitä suositellaan käytettäväksi aina kun on mahdollista, mutta harjoituksissa sen käyttöä on vältetty suorituskyvyn vuoksi.

### Putkittaminen
Ristivalidoinnissa datan esikäsittely, joka riippuu datasta, on hyvä suorittaa erikseen jokaiselle jaolle. Tämä on helpoin tehdä käyttämällä putkitusta ([pipelines](https://scikit-learn.org/stable/modules/compose.html#pipeline)). Sillä tavoin on mahdollista kytkeä paljon esikäsittelyvaiheita yhteen ja suorittaa vaiheet jokaisen jaon kohdalla. Putkitusta on syytä käyttää myös hyperparametrioptimoinnin kanssa.

Suorita ensin äskeinen ristivalidointi uudestaan käyttäen moduulia [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) vaiheita [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) ja [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html), sekä haluaamasi luokittelualgoritmia. 


In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC

pipe = Pipeline([("standard_scaler", StandardScaler()), ("pca", PCA()), ("clf", SVC(gamma=0.01))])
scores = cross_val_score(pipe, X_train, y_train, cv=5, n_jobs=-1)
print("Pisteet ovat:")
for oa in scores:
    print("    OA: {:.2f}".format(oa))

Pisteet ovat:
    OA: 0.97
    OA: 0.97
    OA: 0.97
    OA: 0.99
    OA: 0.96


Suorita sama uudestaan käyttäen helppokäyttöisempää metodia [make_pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html#sklearn.pipeline.make_pipeline)

In [20]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(StandardScaler(), PCA(), SVC(gamma=0.01))
scores = cross_val_score(pipe, X_train, y_train, cv=5, n_jobs=-1)
print("Pisteet ovat:")
for oa in scores:
    print("    OA: {:.2f}".format(oa))

Pisteet ovat:
    OA: 0.97
    OA: 0.97
    OA: 0.97
    OA: 0.99
    OA: 0.96


Lue halutessasi lisää ristivalidoinnista [täältä](https://scikit-learn.org/stable/modules/cross_validation.html).